In [1]:
import os
os.chdir('..')

import types

from puzzle_generator import *

import numpy as np
import urllib.request as urllib
import cv2

In [2]:
width = 15
height = 15

my_img_url = "https://cdn-icons-png.flaticon.com/512/1/1600.png"

num_cells = width*height
min_filled_cells = int(num_cells*0.35)
max_filled_cells = int(num_cells*0.50)

puzzle = RectangularPuzzle(
    width=width,
    height=height
)

max_h_nums = 3
max_v_nums = 3

puzzle.solving_timeout = 60

puzzle.naming = {
    None: '_',
    'f': 'X',
    'e': '.',
    'n': ' ',
}

puzzle.domain_program += f"""
    non_null_value(f).
    non_null_value(e).
    value(V) :- non_null_value(V).
    value(n).
    
    #const max_h_nums={max_h_nums}.
    h_num_index(1..max_h_nums).
    #const max_v_nums={max_v_nums}.
    v_num_index(1..max_v_nums).
    num_value(0..board_width).
    num_value(0..board_height).
"""

puzzle.puzzle_gen_program += """
    1 { puzzle(h_num(R,I),N) : num_value(N) } 1 :-
        row(R), h_num_index(I).
    
    :- puzzle(h_num(R,I),0),
        puzzle(h_num(R,I-1),N), N > 0.
        
    1 { puzzle(v_num(C,I),N) : num_value(N) } 1 :-
        col(C), v_num_index(I).
    
    :- puzzle(v_num(C,I),0),
        puzzle(v_num(C,I-1),N), N > 0.
    
    :- row(R), puzzle(h_num(R,I),0) : h_num_index(I).
    
    1 { addition(h_num(R,I),N) : num_value(N) } 1 :-
        row(R), h_num_index(I).
    :- #sum { N,R,I : addition(h_num(R,I),N) } != 1.
    1 { addition(v_num(C,I),N) : num_value(N) } 1 :-
        col(C), v_num_index(I).
    :- #sum { N,C,I : addition(v_num(C,I),N) } != 1.
        
    #show guessed_number(h_num(R,I),N+M) : puzzle(h_num(R,I),N), addition(h_num(R,I),M).
    #show guessed_number(v_num(R,I),N+M) : puzzle(v_num(R,I),N), addition(v_num(R,I),M).
"""
# puzzle.puzzle_constraints_program += f"""
#     %#minimize {{ 1,C : solution(C,f) }}.
# """

puzzle.solution_program = """
    1 { solution(C,V) : non_null_value(V) } 1 :- cell(C).
    
    %%% row_parse(Row,Cell_index,Num_index,Buffer)
    row_parse(Row,0,0,0) :- row(Row).
    row_parse(Row,0,N,0) :- row(Row),
        puzzle(h_num(Row,N),0).
    row_parse(Row,C+1,N,B+1) :- row(Row),
        row_parse(Row,C,N,B), solution(c(Row,C+1),f).
    row_parse(Row,C+1,N+1,0) :- row(Row),
        row_parse(Row,C,N,B), solution(c(Row,C+1),e),
        puzzle(h_num(Row,N+1),B).
    row_parse(Row,C+1,N,0) :- row(Row),
        row_parse(Row,C,N,0), solution(c(Row,C+1),e).
    row_parse(Row,C,N+1,0) :- row(Row),
        row_parse(Row,C,N,B), C = board_width,
        puzzle(h_num(Row,N+1),B).
    :- row(Row), not row_parse(Row,board_width,max_h_nums,0).
    
    %%% col_parse(Col,Cell_index,Num_index,Buffer)
    col_parse(Col,0,0,0) :- col(Col).
    col_parse(Col,0,N,0) :- col(Col),
        puzzle(v_num(Col,N),0).
    col_parse(Col,C+1,N,B+1) :- col(Col),
        col_parse(Col,C,N,B), solution(c(C+1,Col),f).
    col_parse(Col,C+1,N+1,0) :- col(Col),
        col_parse(Col,C,N,B), solution(c(C+1,Col),e),
        puzzle(v_num(Col,N+1),B).
    col_parse(Col,C+1,N,0) :- col(Col),
        col_parse(Col,C,N,0), solution(c(C+1,Col),e).
    col_parse(Col,C,N+1,0) :- col(Col),
        col_parse(Col,C,N,B), C = board_height,
        puzzle(v_num(Col,N+1),B).
    :- col(Col), not col_parse(Col,board_width,max_v_nums,0).
"""

In [3]:
resp = urllib.urlopen(my_img_url)
image = np.asarray(bytearray(resp.read()), dtype="uint8")
image = cv2.imdecode(image, cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (width, height), interpolation=cv2.INTER_CUBIC)

for y in range(height):
    for x in range(width):
        puzzle.puzzle_constraints_program += \
            f"score(c({y+1},{x+1}),f,{255-image[y][x]}).\n"
        puzzle.puzzle_constraints_program += \
            f"score(c({y+1},{x+1}),e,{image[y][x]}).\n"

puzzle.puzzle_constraints_program += """
    #maximize { Score,C : solution(C,V), score(C,V,Score) }.
"""

In [4]:
puzzle.generate(
    verbose=True,
    precompute_solution=False,
)

Done grounding..
.....
Solving time: 171.74 seconds



In [5]:
def custom_repr_puzzle(self, show_solution=False):
    puzzle_repr = ""
    
    row_nums = {}
    for row in range(1, height+1):
        row_nums[row] = [
            self.puzzle_numbers[f"h_num({row},{index})"]
            for index in range(1, max_h_nums+1)
            if self.puzzle_numbers[f"h_num({row},{index})"] != 0
        ]
    col_nums = {}
    for col in range(1, width+1):
        col_nums[col] = [
            self.puzzle_numbers[f"v_num({col},{index})"]
            for index in range(1, max_v_nums+1)
            if self.puzzle_numbers[f"v_num({col},{index})"] != 0
        ]
    
    if show_solution:
        original_pretty_repr = []
        for row in range(1, self.board_height + 1):
            original_pretty_repr.append(
                "|" + "|".join([
                    f"{self.pretty_name(self.solution[(row, col)]):2}"
                    for col in range(1, self.board_width + 1)
                ]) + "|"
            )
    else:
        original_pretty_repr = []
        for row in range(height):
            original_pretty_repr.append("|  "*(width)+"|")
    
    for top_row in range(max_v_nums):
        puzzle_repr += " "*(3*max_h_nums+1)
        for col in col_nums:
            nums = col_nums[col]
            idx = top_row - (max_v_nums - len(nums))
            if idx < 0:
                puzzle_repr += f"   "
            else:
                puzzle_repr += f"{nums[idx]:<2} "
        puzzle_repr += "\n"
    
    for row in range(1, height+1):
        nums = row_nums[row]
        puzzle_repr += " "*(3*(max_h_nums-len(nums)))
        puzzle_repr += "".join(map(
            lambda n: f"{n:<2} ",
            nums
        ))
        puzzle_repr += original_pretty_repr[row-1]
        puzzle_repr += "\n"
    
    return puzzle_repr

puzzle.pretty_repr_puzzle = types.MethodType(
    custom_repr_puzzle,
    puzzle
)
puzzle.pretty_repr_solution = types.MethodType(
    lambda self: custom_repr_puzzle(self, show_solution=True),
    puzzle
)

In [6]:
print(puzzle.pretty_repr_puzzle())

                                                          1           
          2                       10 11 9  1  3  2     11 1           
          1  2  5  8  10 11 13 13 2  1  1  10 10 14 17 2  6  7  4  4  
      1  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |
      1  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |
      4  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |
   1  1  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |
      4  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |
      5  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |
5  3  1  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |
      14 |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |
      16 |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |
      17 |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |
      17 |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |
      

In [7]:
print(puzzle.pretty_repr_solution())

                                                          1           
          2                       10 11 9  1  3  2     11 1           
          1  2  5  8  10 11 13 13 2  1  1  10 10 14 17 2  6  7  4  4  
      1  |. |. |. |. |. |. |. |. |. |. |. |. |. |. |. |. |X |. |. |. |
      1  |. |. |. |. |. |. |. |. |. |. |. |. |. |X |. |. |. |. |. |. |
      4  |. |. |. |. |. |. |. |. |. |. |. |X |X |X |X |. |. |. |. |. |
   1  1  |. |. |. |. |. |. |. |. |. |. |. |. |X |. |X |. |. |. |. |. |
      4  |. |. |. |. |. |. |. |. |. |. |. |. |X |X |X |X |. |. |. |. |
      5  |. |. |. |. |. |. |. |. |. |. |. |. |. |X |X |X |X |X |. |. |
5  3  1  |. |. |. |. |. |. |X |X |X |X |X |. |. |X |X |X |. |X |. |. |
      14 |. |. |. |. |. |X |X |X |X |X |X |X |X |X |X |X |X |X |X |. |
      16 |. |. |. |. |X |X |X |X |X |X |X |X |X |X |X |X |X |X |X |X |
      17 |. |. |. |X |X |X |X |X |X |X |X |X |X |X |X |X |X |X |X |X |
      17 |. |. |. |X |X |X |X |X |X |X |X |X |X |X |X |X |X |X |X |X |
      